In [ ]:
##Data
import numpy as np
N=4 #Number of subcarriers
L=2 #number of symbols
M=8
k = int(np.log2(M)) 
Data = np.random.randint(2,size=(N*L*k))
print("data = \n" ,Data)

data = 
 [0 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 1 0]


In [ ]:
import numpy as geek
  
# input array
in_arr = geek.array([ 2, 0,  1, 5, 4, 1, 9])
out_arr = geek.argsort(in_arr)
print ("Output sorted array indices : ", out_arr)

Output sorted array indices :  [1 2 5 0 4 3 6]


In [ ]:
!pip install ModulationPy 

In [ ]:
modem = PSKModem(M,gray_map=True,bin_input=True) 
modem.modulate(Data)

NameError: ignored

In [ ]:
def IFFTformatrix(matrix):
    [raws,columns]=matrix.shape
    Data_ifft = np.zeros((N,L), dtype=complex)
    p=0
    while p<columns:
          Data_ifft[:,p]= np.fft.ifft(matrix[:,p])
          p+=1
    return Data_ifft

In [ ]:
print(IFFTformatrix(Data))

[[ 1.75+0.j   2.25+0.j ]
 [-0.75-0.5j -0.25+0.j ]
 [ 0.75+0.j   2.25+0.j ]
 [-0.75+0.5j -0.25+0.j ]]


In [ ]:
#IFFT
p=0
Data_ifft = np.zeros((N,L), dtype=complex)  #per column
while p<L:
      Data_ifft[:,p]= np.fft.ifft(Data[:,p])
      p+=1
print(Data_ifft)

#Data_ifft = np.fft.ifft(Data)
data_final = Data_ifft

[[ 1.75+0.j   2.25+0.j ]
 [-0.75-0.5j -0.25+0.j ]
 [ 0.75+0.j   2.25+0.j ]
 [-0.75+0.5j -0.25+0.j ]]


In [ ]:
#CP

Channel=[1,0.5,0.25]
#length of cycle prefix must be >= length of channel
CP = Data_ifft[N-3:N,:]
#print(CP)
Data_CP = np.vstack((CP, Data_ifft))
print(Data_CP)
data_final = Data_CP

[[-0.75-0.5j -0.25+0.j ]
 [ 0.75+0.j   2.25+0.j ]
 [-0.75+0.5j -0.25+0.j ]
 [ 1.75+0.j   2.25+0.j ]
 [-0.75-0.5j -0.25+0.j ]
 [ 0.75+0.j   2.25+0.j ]
 [-0.75+0.5j -0.25+0.j ]]


In [ ]:
#P/S with CP
i = 0
s = 0
t = 0
serial_data = np.zeros(((N+3)*L), dtype=complex)
while i < L:
      k=0
      while k < N+3: 
            g=(data_final[k,i])
            serial_data[t]=g
            t+=1
            k+=1
      i+=1    
print(serial_data)

[-0.75-0.5j  0.75+0.j  -0.75+0.5j  1.75+0.j  -0.75-0.5j  0.75+0.j
 -0.75+0.5j -0.25+0.j   2.25+0.j  -0.25+0.j   2.25+0.j  -0.25+0.j
  2.25+0.j  -0.25+0.j ]


In [ ]:
#sizes
print(Data_ifft.shape)
print(Data_CP.shape)
print(serial_data.shape)

(4, 2)
(7, 2)
(14,)


In [ ]:
#PAPR
PAPR1 = np.convolve(serial_data,np.conj(serial_data))   #conj
max=np.max(PAPR1)
mean=np.mean(PAPR1)
print(max, mean)
PAPR=max/mean
print(PAPR)



(16.125+0j) (1.3333333333333333+0j)
(12.09375+0j)


In [ ]:
#convolution between data and channel
from scipy.ndimage import convolve
data_rx = np.convolve(serial_data,Channel)
print(data_rx)
print(data_rx.shape)

[-0.75  -0.5j    0.375 -0.25j  -0.5625+0.375j  1.5625+0.25j
 -0.0625-0.375j  0.8125-0.25j  -0.5625+0.375j -0.4375+0.25j
  1.9375+0.125j  0.8125+0.j     2.6875+0.j     0.8125+0.j
  2.6875+0.j     0.8125+0.j     0.4375+0.j    -0.0625+0.j   ]
(16,)


In [ ]:
#reshaping data with CP
data_rx1 = data_rx[0:95]
#print(data_rx1)

i = 0
s = 0
t=0
reshaped_data = np.zeros(((N+3),L), dtype=complex)
while i < L:
      k=0
      while k < N+3: #19
            g=(data_rx1[t])
            reshaped_data[k,i]=g
            t+=1
            k+=1
      i+=1    
print(reshaped_data)
print(reshaped_data.shape)
#print(data_rx1.shape)
#data_rx_reshaped=data_rx1.reshape((N+3),L)
#print(data_rx_reshaped)
#print(data_rx_reshaped.shape)

[[-0.75  -0.5j   -0.4375+0.25j ]
 [ 0.375 -0.25j   1.9375+0.125j]
 [-0.5625+0.375j  0.8125+0.j   ]
 [ 1.5625+0.25j   2.6875+0.j   ]
 [-0.0625-0.375j  0.8125+0.j   ]
 [ 0.8125-0.25j   2.6875+0.j   ]
 [-0.5625+0.375j  0.8125+0.j   ]]
(7, 2)


In [ ]:
#Removing cycle Prefix
data_without_CP = reshaped_data[3:N+3,:]
print(data_without_CP)

[[ 1.5625+0.25j   2.6875+0.j   ]
 [-0.0625-0.375j  0.8125+0.j   ]
 [ 0.8125-0.25j   2.6875+0.j   ]
 [-0.5625+0.375j  0.8125+0.j   ]]


In [ ]:
#FFT
import scipy as sp

data_without_CP_or_fft = np.zeros((N,L), dtype=complex)

p=0
while p<L:
      data_without_CP_or_fft[:,p]= np.fft.fft(data_without_CP[:,p])
      p+=1
print(data_without_CP_or_fft)
print(data_without_CP_or_fft.shape)

[[1.75+0.j 7.  +0.j]
 [0.  +0.j 0.  +0.j]
 [3.  +0.j 3.75+0.j]
 [1.5 +1.j 0.  +0.j]]
(4, 2)


In [ ]:
#channel equalyzier
import numpy as np
from numpy import matrix
from numpy import linalg
#data_recover = np.zeros((N,L), dtype=complex)
data_recover1 = np.zeros((N,L), dtype=complex)
H_transpose = np.zeros((16,1), dtype=complex)
H= np.fft.fft(Channel,N)
#H_transpose=H.transpose()
k=0
while k < N:
      g=(H[k])
      H_transpose[k,0]=g
      k+=1
#print(H_transpose)
#print(H)
#print(H_transpose)
#print(H.shape)
#print(H_transpose.shape)
#H_real=H.real
#print(H_real)
#print(data_without_CP_or_fft)
i=0
while i<L:
      #data_recover[:,i]=np.divide(data_without_CP_or_fft[:,i],H_real)
      data_recover1[:,i]=data_without_CP_or_fft[:,i]/H
      i+=1
#data_recover2=data_recover1.real
print(data_recover1)
#print(data_recover)
#print(data_recover.shape)

[[1.+0.j 4.+0.j]
 [0.+0.j 0.+0.j]
 [4.+0.j 5.+0.j]
 [2.+0.j 0.+0.j]]


In [ ]:
print(Data[:,1],data_recover1[:,1])

[4 0 5 0] [4.+0.j 0.+0.j 5.+0.j 0.+0.j]
